In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn

In [25]:
df_original = pd.read_csv("results/hochuli/new_count.csv")
df_double = pd.read_csv("results/hochuli_double/new_count.csv")
df_deep = pd.read_csv("results/hochuli_deep/new_count.csv")
df_mobilenet = pd.read_csv("results/mobilenet/count.csv")

In [26]:
# fazendo a média por execução (cada época aparece num_execucoes)
df_original = df_original.groupby("epoch").mean()
df_double = df_double.groupby("epoch").mean()
df_deep = df_deep.groupby("epoch").mean()
df_mobilenet = df_mobilenet.groupby("epoch").mean()

In [27]:
df_deep.head(5)

,execution,train_loss,val_loss,val_acc,time,zero_relu_call,total_relu_call
epoch,,,,,,,
-1,3.0,NaN,0.322473,0.886260,1.136197,0.0,0.000000e+00
1,3.0,2.303302,2.302398,0.100000,16.913589,229.6,1.056384e+09
2,3.0,2.301682,2.300086,0.153278,16.969304,283.2,1.056384e+09
3,3.0,2.068055,1.702323,0.372189,16.919213,1994.0,1.056384e+09
4,3.0,1.644859,1.396304,0.481533,16.921069,1804.8,1.056384e+09


In [28]:
# variáveis de interesse
dados = {"original": df_original, "double": df_double, "deep": df_deep, "mobilenet": df_mobilenet}
resultados = {}
for nome, df in dados.items():
    relu0 = round(df["zero_relu_call"].sum())
    relux = round(df["total_relu_call"].sum())
    prop = relu0 / relux
    resultados[nome] = [relu0, relux, prop]

In [29]:
from model_utils import Hochuli, HochuliDeep, HochuliDoubleKernels, create_mobile_net
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

resultados["original"].append(count_parameters(Hochuli([3, 28, 28], 10, nn.ReLU())))
resultados["double"].append(count_parameters(HochuliDoubleKernels([3, 28, 28], 10, nn.ReLU())))
resultados["deep"].append(count_parameters(HochuliDeep([3, 28, 28], 10, nn.ReLU())))
resultados["mobilenet"].append(count_parameters(create_mobile_net(nn.ReLU(), nn.Hardswish(), nn.Hardsigmoid(), 10, pre_trained=True, freeze=False)))

In [30]:
linhas = ["ReLU'(0)", "ReLU'(x)", "ReLU'(0)/ReLU'(x)", "Parâmetros"]
pd.DataFrame(data=resultados, index=linhas)

,original,double,deep,mobilenet
ReLU'(0),4.038500e+05,8.788240e+05,2.335500e+04,2.580000e+03
ReLU'(x),5.281920e+10,1.055040e+11,5.281920e+10,5.779200e+10
ReLU'(0)/ReLU'(x),7.645894e-06,8.329769e-06,4.421688e-07,4.464286e-08
Parâmetros,7.341800e+04,2.567140e+05,1.104420e+05,1.528106e+06


In [31]:
resultados["original"][2] / resultados["deep"][2]

17.291800470991223

In [32]:
resultados["original"][2] / resultados["mobilenet"][2]

171.26802374893978